In [1]:
# pip install pandas
import pandas as pd

from collections import Counter
from operator import itemgetter
from datetime import datetime

In [2]:
df = pd.read_csv("shkib.csv")
df.head()

,_time,src_user,src_ip,src_port,dest_user,dest_ip,dest_port,input_byte,output_byte
0,2018-01-19T19:46:00.000+0300,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55991,NaN,1f1b55833917801cbd2ed9a313095d9e,443,1019,3562
1,2018-01-19T19:46:00.000+0300,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55979,NaN,353a9f7687529bc37e3c8bcae6dcfdfa,443,840,357
2,2018-01-19T19:46:00.000+0300,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,34432,NaN,072f83db38ddd013e59a3616bd3b05a6,53,61,135
3,2018-01-19T19:46:00.000+0300,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,49932,NaN,072f83db38ddd013e59a3616bd3b05a6,53,61,77
4,2018-01-19T19:46:00.000+0300,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55994,NaN,ee76d4dce963e690712339f7bf064705,443,757,3511


### Поиск 5ти пользователей, сгенерировавших наибольшее количество запросов

In [3]:
def topRequestUser(df):
    groups = df[df["src_user"].notnull()].groupby(by=["src_user"]).groups
    user_cnt_queries = map(lambda (user, queries): (user, len(queries)), groups.items())
    topRequest = sorted(user_cnt_queries, key=itemgetter(1),reverse=True)[:5]
    return topRequest

In [4]:
topRequestUser(df)

[('a6c732a9cd426b7620cd93e7aa37aef6', 54522),
 ('7834ea899abe21b5fd047e125cdffc56', 50149),
 ('64c5eae485561a1d7064477e792a6358', 46463),
 ('aedf2485753a52ca1149bde2343e8d7a', 40371),
 ('e948e29d93d507d2bfd6dc2dee45bf2b', 37718)]

### Поиск 5ти пользователей, отправивших наибольшее количество данных

In [5]:
def topDataSendUser(df):
    groups = df[df["src_user"].notnull()].groupby(by=["src_user"])['output_byte']
    topDataSend = sorted(dict(groups.sum()).items(), key=itemgetter(1),reverse=True)[:5]
    return topDataSend

In [6]:
topDataSendUser(df)

[('ef407a4a8c7262284b558651b8e7eff3', 5788762342),
 ('22e94d0520b9f9b2f02eaa97b165f720', 2695498079),
 ('453370ebc5e3c98cc53664362da6cc1a', 1560165694),
 ('34902a8b61bc416b1ff8832d7c3bbddb', 1560156359),
 ('c15cf96d9b56740c974661d209ef44f7', 1272035251)]

### Поиск регулярных запросов (запросов выполняющихся периодически) по полю src_user

In [7]:
def getTop10RegularUsr(df):
    df_without_nan_user = df[df["src_user"].notnull()]
    
    columns = ['src_user', 'dest_ip', 'output_byte', 'dest_port']
    df_string = df_without_nan_user[columns].astype(str)
    cnt = Counter(map(lambda x: ' '.join(x), df_string.values))
    cnt = sorted(cnt.items(), key=itemgetter(1), reverse=True)
     
    top = cnt[:10]
    data = map(lambda (data, cnt): data.split() + [cnt], top)
    return pd.DataFrame(data=data, columns=columns + ['count_queries'], index=None)

In [8]:
topRegular = getTop10RegularUsr(df)
topRegular

,src_user,dest_ip,output_byte,dest_port,count_queries
0,7834ea899abe21b5fd047e125cdffc56,072f83db38ddd013e59a3616bd3b05a6,94,53,9419
1,7834ea899abe21b5fd047e125cdffc56,072f83db38ddd013e59a3616bd3b05a6,123,53,8019
2,f2825c1deb48cc723dc27b598c662610,0956783fe5a6d8e0f864b5eaf4084063,123,53,5863
3,7834ea899abe21b5fd047e125cdffc56,072f83db38ddd013e59a3616bd3b05a6,136,53,5725
4,7834ea899abe21b5fd047e125cdffc56,072f83db38ddd013e59a3616bd3b05a6,106,53,4300
5,0621b1f7a35e6e0617dc82d86edb1759,0956783fe5a6d8e0f864b5eaf4084063,0,53,3639
6,f2825c1deb48cc723dc27b598c662610,072f83db38ddd013e59a3616bd3b05a6,52,53,2944
7,f2825c1deb48cc723dc27b598c662610,0956783fe5a6d8e0f864b5eaf4084063,52,53,2934
8,f2825c1deb48cc723dc27b598c662610,0956783fe5a6d8e0f864b5eaf4084063,114,53,2931
9,a6c732a9cd426b7620cd93e7aa37aef6,0956783fe5a6d8e0f864b5eaf4084063,0,53,2844


### Поиск регулярных запросов (запросов выполняющихся периодически) по полю src_ip

In [9]:
def getTop10RegularIP(df):
    
    columns = ['src_ip', 'dest_ip', 'output_byte', 'dest_port']
    df_string = df[columns].astype(str)
    cnt = Counter(map(lambda x: ' '.join(x), df_string.values))
    cnt = sorted(cnt.items(), key=itemgetter(1), reverse=True)
     
    top = cnt[:10]
    data = map(lambda (data, cnt): data.split() + [cnt], top)
    return pd.DataFrame(data=data, columns=columns + ['count_queries'], index=None)

In [10]:
toRegularIP = getTop10RegularIP(df)
toRegularIP

,src_ip,dest_ip,output_byte,dest_port,count_queries
0,d02c9bb2cade1c33ff985188acc96e68,072f83db38ddd013e59a3616bd3b05a6,94,53,9419
1,d02c9bb2cade1c33ff985188acc96e68,072f83db38ddd013e59a3616bd3b05a6,123,53,8019
2,43ed6a1c781031fa618c0350a801e52f,0956783fe5a6d8e0f864b5eaf4084063,123,53,5863
3,d02c9bb2cade1c33ff985188acc96e68,072f83db38ddd013e59a3616bd3b05a6,136,53,5725
4,d02c9bb2cade1c33ff985188acc96e68,072f83db38ddd013e59a3616bd3b05a6,106,53,4300
5,b69be4fe5400cb92f22f9921ef73b8dc,0956783fe5a6d8e0f864b5eaf4084063,0,53,3639
6,43ed6a1c781031fa618c0350a801e52f,072f83db38ddd013e59a3616bd3b05a6,52,53,2944
7,43ed6a1c781031fa618c0350a801e52f,0956783fe5a6d8e0f864b5eaf4084063,52,53,2934
8,43ed6a1c781031fa618c0350a801e52f,0956783fe5a6d8e0f864b5eaf4084063,114,53,2931
9,3f491769846e92a9e9e1afec2321b5ae,0956783fe5a6d8e0f864b5eaf4084063,0,53,2844
